# 中性原子量子コンピュータ上の単位円グラフを用いたQUBO/イジング問題の解法

## 依存関係のインストール

In [ ]:
# !pip install bloqade-analog

## 必要なライブラリのインポート

In [ ]:
from itertools import product
import math

from bloqade.analog import start
import jijmodeling as jm
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoMinorLocator, FormatStrFormatter
import networkx as nx
import numpy as np

from qamomile.core.converters.qaoa import QAOAConverter
from qamomile.udm import Ising_UnitDiskGraph
from qamomile.udm.transpiler import UDMTranspiler

## 問題ハミルトニアンの定式化  

### `Jijmodeling`による問題の定式化
以下では、`Jijmodeling` を使用して問題ハミルトニアンを定式化する方法を示します。

In [ ]:
def QUBO_problem():
    # --- プレースホルダー ---
    V = jm.Placeholder("V")
    E = jm.Placeholder("E", ndim=2)
    U = jm.Placeholder("U", ndim=2)

    n = jm.BinaryVar("n", shape=(V,))
    e = jm.Element("e", belong_to=E)

    # --- 問題の定式化 ---
    problem = jm.Problem("QUBO_Hamiltonian")

    quadratic_term = jm.sum(e, U[e[0], e[1]] * n[e[0]] * n[e[1]])
    problem += quadratic_term
    return problem


problem = QUBO_problem()
problem

In [ ]:
quad = {
    (0, 0): -1.2,
    (1, 1): -3.2,
    (2, 2): -1.2,
    (0, 1): 4.0,
    (0, 2): -2.0,
    (1, 2): 3.2,
}

V_val = 3
E_val = np.array(list(quad.keys()), dtype=int)
U_val = np.zeros((V_val, V_val))
for (i, j), Jij in quad.items():
    U_val[i, j] = U_val[j, i] = Jij

instance = {
    "V": V_val,
    "E": E_val,
    "U": U_val,
}

compiled_instance = jm.Interpreter(instance).eval_problem(problem)

### `qamomile` を使った問題の変換

`compiled_instance` を取得したら、`QuantumConverter` を使用してそれを `IsingModel` クラスに変換することができます。

In [ ]:
udm_converter = QAOAConverter(compiled_instance)
ising_model = udm_converter.ising_encode()
ising_model

## `qamomile` によるユニットディスクグラフへのマッピング

### マッピングの実行: `UnitDiskGraph` クラス

`UnitDiskGraph` クラスは、`IsingModel` をユニットディスクグラフ（UDG）表現に変換する役割を担います。

In [ ]:
udg = Ising_UnitDiskGraph(ising_model)

print(f"Converted to Unit Disk Graph:")
print(f"- {len(udg.nodes)} nodes in the grid graph")
print(f"- {len(udg.pins)} pins (nodes corresponding to original variables)")

### UDG の可視化

`networkx` と `matplotlib` を用いることで、変換後のユニットディスクグラフ（UDG）構造を視覚的に確認できます。  
`UnitDiskGraph` オブジェクトは、基礎となるグラフ構造へのアクセスを提供します。  
この可視化により、抽象的な Ising 問題がユニットディスクマッピング（UDM）ガジェットを使って物理的にどのように配置されているかを理解するのに役立ちます。

In [ ]:
G_vis = udg.networkx_graph
pos = nx.get_node_attributes(G_vis, "pos")
pins = udg.pins
plt.figure(figsize=(5, 4))

# ハイライトピン（元の変数）
node_colors = ["Green" if i in pins else "lightblue" for i in G_vis.nodes()]
node_sizes = [300 if i in pins else 100 for i in G_vis.nodes()]

nx.draw_networkx_nodes(G_vis, pos, node_color=node_colors, node_size=node_sizes)
nx.draw_networkx_edges(G_vis, pos, width=0.5, alpha=0.5)

# ピンに元の変数のインデックスをラベル付けする
pin_labels = {pin: f"z{i}" for i, pin in enumerate(pins)}  # Use z_i for Ising spins
nx.draw_networkx_labels(G_vis, pos, labels=pin_labels, font_size=10)

plt.title("Unit Disk Graph Representation of the Ising Model", fontsize=15)
plt.xlabel("X coordinate", fontsize=15)
plt.ylabel("Y coordinate", fontsize=15)
plt.axis("equal")  # Ensure aspect ratio is maintained
plt.tight_layout()
plt.show()

## MWIS-UDG を **`bloqade-analog`** で解く

Ising 問題をユニットディスクグラフ（UDG）上の MWIS（最大重み独立集合）問題に変換できたので、  
中性原子量子コンピュータのネイティブ機能を活用して解を求めることができます。

### **`bloqade-analog`** を用いた実装

`bloqade-analog` は、中性原子量子計算、特にアナログ方式や断熱プロトコルのシミュレーションに特化した Python ライブラリです。  
Qamomile で生成した UDG マッピングに対して `bloqade_example.py` スクリプトを使ってこれを活用できます。

**ステップ:**

1. **入力の準備:**
   - 原子の配置座標を `UnitDiskGraph` オブジェクト (`udg.nodes`) から取得します。これらの位置は、中性原子系における typical なブロッケード半径に合わせて物理単位（例：μm）にスケーリングする必要があります。
   - 各ノードの重み `udg.nodes[i].weight` を取得します（通常は正規化された値を使用）。
   - パルスパラメータを定義します：
     - 最大ラビ周波数 `Omega_max`
     - 最大デチューニング `delta_max`
     - 進化時間 `t_max`

In [ ]:
LOCATION_SCALE = 5.0  # 希望するブロケード半径とハードウェアに基づいて調整する

locations = udg.qubo_result.qubo_grid_to_locations(LOCATION_SCALE)
weights = udg.qubo_result.qubo_result_to_weights()

print(f"Node weights: {weights}")
print(f"Node locations: {locations}")

2.  **プログラムの定義：** `bloqade.analog` を使って原子の配置とパルスシーケンスを定義する。

In [ ]:
def solve_ising_bloqade(locations, weights, delta_max=60.0, Omega_max=15.0, t_max=4.0):
    locations_array = np.array(locations)
    centroid = locations_array.mean(axis=0)
    centered_locations = locations_array - centroid
    locations = list(map(tuple, centered_locations))

    lw = len(weights)
    weights_norm = [x / max(weights) for x in weights]

    def sine_waveform(t):
        return Omega_max * math.sin(math.pi * t / t_max) ** 2

    def linear_detune_waveform(t):
        return delta_max * (2 * t / t_max - 1)

    program = (
        start.add_position(locations)
        .rydberg.detuning.scale(weights_norm)
        .fn(linear_detune_waveform, t_max)
        .amplitude.uniform.fn(sine_waveform, t_max)
    )

    return program


program = solve_ising_bloqade(locations, weights)

3.  **シミュレーションの実行：** Bloqade のエミュレータを使ってプログラムを実行する。

In [ ]:
blockade_radius = (
    LOCATION_SCALE * 1.5
)  # ブロケード半径は対角ノードをカバーするように設定されている（1：√2）
emu_results = program.bloqade.python().run(
    shots=10000, solver_name="dop853", blockade_radius=blockade_radius
)

report = emu_results.report()
counts = report.counts()[0]

sorted_counts = {
    k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)
}

print("\nBloqade Simulation Results (Top 5):")
for i, (bitstring, count) in enumerate(list(sorted_counts.items())[:5]):
    print(f" {i+1}. Bitstring: {bitstring} (Count: {count})")

### Bloqade のエミュレータからの結果の可視化

Bloqade のエミュレータから得られた結果を可視化することができる。赤いノードは `Bitstring` のマッピングを示している。

In [ ]:
G_vis = udg.networkx_graph
pos = nx.get_node_attributes(G_vis, "pos")

plt.figure(figsize=(5, 4))

bitstr = list(sorted_counts.items())[:2][0][0]

# ビット値に基づいてノードの色とサイズを変更する
node_colors = ["red" if b == "0" else "lightgray" for b in bitstr]
node_sizes = [300 if i in pins else 100 for i in G_vis.nodes()]

nx.draw_networkx_nodes(G_vis, pos, node_color=node_colors, node_size=node_sizes)
nx.draw_networkx_edges(G_vis, pos, width=0.5, alpha=0.5)

# ピンに元の変数のインデックスをラベル付けする
pin_labels = {pin: f"z{i}" for i, pin in enumerate(pins)}  # 偉人ぐスピンとして z_i を使う
nx.draw_networkx_labels(G_vis, pos, labels=pin_labels, font_size=10)

plt.title("Unit Disk Graph Representation of the Ising Model", fontsize=15)
plt.xlabel("X coordinate", fontsize=15)
plt.ylabel("Y coordinate", fontsize=15)
plt.axis("equal")  # アスペクト比が維持されるようにする
plt.tight_layout()
plt.show()

### Bloqade のエミュレータ結果を `sampleset` 結果に変換する

先に得られたエミュレータの結果から、`udm_converter.decode` を使ってそれらを sampleset に変換することができる。`sampleset` を用いることで、目的関数値の分布を調べることができる。

In [ ]:
transpiler = UDMTranspiler(udg, V_val)
sampleset = udm_converter.decode(transpiler, counts)

### 最も低い解を確認する

In [ ]:
sampleset.lowest()

### 古典的手法を確認する

In [ ]:
Q = np.array([[-1.2, 4.0, -2.0], [4.0, -3.2, 3.2], [-2.0, 3.2, -1.2]])


def Solve_Q(Q: np.ndarray, x: np.ndarray) -> float:
    E = np.dot(np.diag(Q), x)
    n = len(x)
    for i in range(n):
        for j in range(i + 1, n):
            E += Q[i, j] * x[i] * x[j]
    return E


energies = {}
for bits in product([0, 1], repeat=3):
    x = np.array(bits)
    energy = Solve_Q(Q, x)
    energies[bits] = energy
min_config, min_energy = min(energies.items(), key=lambda item: item[1])
print("min_config: ", min_config)
print("min_energy: ", min_energy)

## 結果の評価

In [ ]:
plot_data = {}
for sample in sampleset:
    if sample.eval.objective in plot_data:
        plot_data[sample.eval.objective] += sample.num_occurrences
    else:
        plot_data[sample.eval.objective] = sample.num_occurrences

energies = sorted(plot_data.keys())
freqs = [plot_data[e] for e in energies]
plt.figure(figsize=(8, 5))
plt.bar(energies, freqs, width=0.25, label="Energies from Sampleset")

ax = plt.gca()
ax.set_xticks(energies)
ax.set_xticklabels([f"{e:.2f}" for e in energies], rotation=45, ha="right")
ax.axvline(min_energy, color="red", linestyle="--", linewidth=2, label="Min energy")

ax.xaxis.set_minor_locator(AutoMinorLocator(4))

plt.title("Distribution of solution", fontsize=15)
plt.ylabel("Frequency", fontsize=15)
plt.xlabel("Energy", fontsize=15)
plt.legend(loc="upper center")
plt.tight_layout()
plt.show()